In [1]:
%run my_import

#uri = socket.gethostname()+".local:5555"
uri = "com-exp.local:5555"
#uri = "localhost:5555"
io = ZmsgIO(uri)

bId = 11
ids = [bId]

homing = {"type": "TRJ_CMD",
          "trajectory_cmd": {"type": "HOMING", "name": "homing", "board_id": bId,
                             "homing_par": {"x": [0,3]}}}

xs = [0,1,2,3,4]
xs = [x*2 for x in xs] 
smooth = {"type": "TRJ_CMD",
             "trajectory_cmd": {"type": "SMOOTHER", "name": "smooth", "board_id": bId,
                                "smooth_par": {"x": xs, "y": [0, 0.5, 0, -0.5, 0]}}} 
sine = {"type": "TRJ_CMD",
        "trajectory_cmd": {"type": "SINE", "name": "sine", "board_id": bId,
                           "period_par": {"freq": 0.2, "ampl": 2.79, "teta": 0, "secs": 500}}}
sine = {"type": "TRJ_CMD",
        "trajectory_cmd": {"type": "SINE", "name": "sine", "board_id": bId,
                           "period_par": {"freq": 0.2, "ampl": 1.0, "teta": 0, "secs": 500}}}

add_homing = {"type": "TRJ_QUEUE_CMD",
              "trj_queue_cmd": {"type": "PUSH_QUE", "trj_names": ["homing"]}}
add_smooth = {"type": "TRJ_QUEUE_CMD",
              "trj_queue_cmd": {"type": "PUSH_QUE", "trj_names": ["smooth"]}}
add_sine =   {"type": "TRJ_QUEUE_CMD",
              "trj_queue_cmd": {"type": "PUSH_QUE", "trj_names": ["sine"]}}
add_all =   {"type": "TRJ_QUEUE_CMD",
              "trj_queue_cmd": {"type": "PUSH_QUE", "trj_names": ["homing","smooth","sine"]}}

io.debug=0

In [31]:
reply = io.doit(ctrl_cmd_set_home.set_bid(bId))

In [28]:
#io.doit(ctrl_cmd_set_zero.set_value(3.14159).set_bid(bId))

{'type': 1, 'cmd_type': 2, 'msg': 'Ok !!!'}

In [32]:
#reply = io.doit(ctrl_cmd_start.set_bid(bId).set_value(0x3B).set_gains(u'POSITION',200,0,15,0,0,0))
reply = io.doit(ctrl_cmd_start.set_bid(bId).set_value(0x3B))

In [34]:
# define trajectories
reply = io.doit(homing)
reply = io.doit(smooth)
reply = io.doit(sine)

In [35]:
#reply = io.doit(add_homing)
#reply = io.doit(add_smooth)
#reply = io.doit(add_sine)
reply = io.doit(add_all)

In [12]:
reply = io.doit(ctrl_cmd_stop.set_bid(bId))

In [ ]:
reply = io.doit(ctrl_cmd_start.set_bid(bId).set_value(0xD4))

In [ ]:
io.debug=1
gains = {}
gains['type'] = "IMPEDANCE"
gains['kp'] = 1
gains['kd'] = 0
gains['ki'] = 0
gains['kkp'] = 5
gains['kkd'] = 0.01
gains['kki'] = 0.8
reply = io.doit(ctrl_cmd_start.set_bid(bId).set_value(0xD4).set_gains(*gains.values()))

In [ ]:
cmd

In [ ]:
xs

In [ ]:
gains.values()

In [33]:
io.doit(ctrl_cmd_fan.set_bid(bId).set_value(1))

{'type': 1, 'cmd_type': 2, 'msg': 'Ok !!!'}